In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
dataPath = "/content/gdrive/MyDrive/levware_test_data.csv"

In [ ]:
# datasets: hugging-face의 인공지능 커뮤니티의 데이터를 불러오고 관리하는 라이브러리
# 기존 라이브러리가 2월19일 기준 업데이트 되면서 pyarrow 라이브러리와 호환성 문제 발생 띠리사 호환라이브러리 정의
# jsonlines: json 포맷의 데이터를 핸들링하는 라이브러리
!pip install datasets==2.16.1 jsonlines==4.0.0

In [ ]:
# hugging face 로그인: hugging face -> 우측상단의 계정선택 -> settings -> access token
import huggingface_hub
huggingface_hub.login()

In [ ]:
# 행과열을 핸들링하는 라이브러리
import pandas as pd
# json 포맷 데이터를 핸들링하는 라이브러리
import json
import jsonlines
# hugging face 데이터 관리 라이브러리
from datasets import Dataset

In [ ]:
# 데이터 경로 설정 및 불러오기
datasetName = "levware_test_data.csv"
jsonFileName = "levware_test_data.jsonl"

In [ ]:
def csv_to_json(csv_file_path, json_file_path):
    # CSV 파일을 DataFrame으로 읽기★ (한글인 경우 encoding="ms949") 영문인경우 해당 내용 삭제 ★
    df = pd.read_csv(csv_file_path, encoding="cp949")

    # JSON 파일로 저장
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        # 각 행을 JSON으로 변환하여 바로 파일에 쓰기
        for index, row in df.iterrows():
            data = {'inputs': row['inputs'], 'response': row['response']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')  # 각 행마다 줄바꿈

# CSV 파일 경로와 JSON 파일 경로 설정
csv_file_path = dataPath
json_file_path = "/content/gdrive/MyDrive/"+jsonFileName

print("csv_file_path: ", csv_file_path)
print("json_file_path: ", json_file_path)

# 함수 호출
csv_to_json(csv_file_path, json_file_path)

In [ ]:
indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
      indataset.append(f'<s>[INST] {line["inputs"]} [/INST] {line["response"]} </s>')
    #   indataset.append(f'<s>### Instruction: \n{line["inputs"]} \n\n### Response: \n{line["response"]}</s>')

# 데이터셋 확인
print('데이터셋 확인')
print(indataset[:5])
print(dataPath)

# 데이터셋 생성 및 저장
indataset = Dataset.from_dict({"text": indataset})
# indataset.save_to_disk(dataPath)

# 데이터셋 info 확인
print('데이터셋 info 확인')
print(indataset)

In [ ]:
print(indataset)
indataset.push_to_hub("kyounghyun/levware_test")